In [1]:
import scipy.io
import os
import numpy as np

In [2]:
# Define the main path and file paths
main_path = r'D:\Fateme A\23_Brain Electrophysiological Recording during Olfactory Stimulation in Mild Cognitive Impairment and Alzheimer Disease Patients An EEG Dataset'

In [3]:
import os
import numpy as np
import scipy.io
import random

# Define file names
file_paths = {
    "AD": os.path.join(main_path, "AD.mat"),
    "normal": os.path.join(main_path, "normal.mat"),
    "MCI": os.path.join(main_path, "MCI.mat")
}

# Prepare storage per class
class_data = {
    "AD": [],
    "normal": [],
    "MCI": []
}

# Load and organize all trials by class
for label, file_path in file_paths.items():
    mat_data = scipy.io.loadmat(file_path)
    eeg_data = mat_data[label]  # Assuming variable name matches label

    for patient_id in range(eeg_data.shape[1]):  # shape: (1, N_patients)
        patient_data = eeg_data[0][patient_id]
        trials = patient_data[0]  # shape: channels x samples x trials

        # Split into individual trials and add to class list
        for i in range(trials.shape[2]):
            class_data[label].append(trials[:, :, i])  # One trial

# Calculate total trial counts per class
trial_counts = {label: len(trials) for label, trials in class_data.items()}
print("Trial counts per class:", trial_counts)

# Find the minimum number of trials across the 3 classes
min_trial_count = min(trial_counts.values())
print("trial number to balance classes:", min_trial_count)

# Randomly select min_trial_count trials from each class
balanced_data = []
balanced_labels = []

for label in class_data:
    selected_trials = random.sample(class_data[label], min_trial_count)
    balanced_data.extend(selected_trials)
    balanced_labels.extend([label] * min_trial_count)

# Convert to arrays
data_array = np.stack(balanced_data)  # shape: total_trials x channels x samples
labels_array = np.array(balanced_labels)

# Final output info
print("3 * 848 = ", data_array.shape[0])
print('we used random selection of trials to balance classes')

print("Final balanced dataset:")
print("Data shape:", data_array.shape)
print("Labels shape:", labels_array.shape)
print("Labels:", np.unique(labels_array, return_counts=True))


Trial counts per class: {'AD': 1352, 'normal': 1677, 'MCI': 848}
trial number to balance classes: 848
3 * 848 =  2544
we used random selection of trials to balance classes
Final balanced dataset:
Data shape: (2544, 4, 600)
Labels shape: (2544,)
Labels: (array(['AD', 'MCI', 'normal'], dtype='<U6'), array([848, 848, 848]))


## Transfer Entropy Function

In [4]:
import numpy as np
from scipy.signal import hilbert
from sklearn.preprocessing import KBinsDiscretizer

def compute_phase(signal):
    """Compute instantaneous phase via Hilbert transform."""
    analytic_signal = hilbert(signal)
    phase = np.angle(analytic_signal)
    return phase

def bin_phases(phases, n_bins):
    """Discretize continuous phases into n_bins states uniformly."""
    # phases in [-π, π]
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform', subsample=None)
    return est.fit_transform(phases.reshape(-1, 1)).astype(int).flatten()

def estimate_joint_probabilities(data, n_bins):
    """Estimate joint probability distribution from 2D array of binned variables."""
    # Each row = variable, columns = samples
    # data.shape = (variables, N)
    variables, N = data.shape
    joint_counts = np.zeros([n_bins]*variables, dtype=np.float64)
    for i in range(N):
        idx = tuple(data[:, i])
        joint_counts[idx] += 1
    joint_probs = joint_counts / joint_counts.sum()
    return joint_probs

def phase_transfer_entropy(signal_a, signal_b, delta=1, n_bins=8):
    """
    Computes Phase Transfer Entropy from signal_b (driver) to signal_a (target)
    using the formula from the image.
    """
    # Step 1: Compute instantaneous phases
    phase_a = compute_phase(signal_a)
    phase_b = compute_phase(signal_b)

    # Step 2: Embed
    # A_delta = future of A
    A_delta = phase_a[delta:]
    A_past = phase_a[:-delta]
    B_past = phase_b[:-delta]

    N_effective = len(A_delta)

    # Step 3: Bin phases
    A_delta_binned = bin_phases(A_delta, n_bins)
    A_past_binned = bin_phases(A_past, n_bins)
    B_past_binned = bin_phases(B_past, n_bins)

    # Step 4: Build joint data
    data_A_delta_A_B = np.vstack([A_delta_binned, A_past_binned, B_past_binned])
    data_A_delta_A = np.vstack([A_delta_binned, A_past_binned])

    # Step 5: Estimate probabilities
    p_A_delta_A_B = estimate_joint_probabilities(data_A_delta_A_B, n_bins)
    p_A_delta_A = estimate_joint_probabilities(data_A_delta_A, n_bins)

    # Step 6: Compute sum
    PTE = 0.0
    for i in range(n_bins):
        for j in range(n_bins):
            for k in range(n_bins):
                p_joint = p_A_delta_A_B[i, j, k]
                if p_joint == 0:
                    continue

                p_cond_AB = p_joint / (p_A_delta_A_B[:, j, k].sum() + 1e-12)
                p_cond_A = p_A_delta_A[i, j] / (p_A_delta_A[:, j].sum() + 1e-12)

                if p_cond_AB > 0 and p_cond_A > 0:
                    PTE += p_joint * np.log2(p_cond_AB / p_cond_A)

    return PTE


## Comparing Classifiers

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import butter, filtfilt

# === Classifiers ===
classifiers = {
    'SVM': SVC(kernel='rbf', random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Naive Bayes': GaussianNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000, solver='lbfgs', C=1.0, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

# Define frequency bands
bands = {
    "delta": (0.1, 4),
    "theta": (4, 8),
    "alpha": (8, 14),
    "beta": (14, 30),
    "gamma": (30, 40)
}

def bandpass_filter(signal, lowcut=0.1, highcut=40.0, fs=200.0, order=3):
    """Apply a Butterworth bandpass filter to a signal."""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

# --- Transfer entropy feature extraction for specific bands ---
def extract_transfer_entropy_features_between_bands(trial, band_range_1, band_range_2):
    """
    Extract phase transfer entropy features between two frequency bands from the same channel.
    trial shape: (channels, samples)
    band_range_1: tuple (low_freq, high_freq) for first band
    band_range_2: tuple (low_freq, high_freq) for second band
    """
    low_freq_1, high_freq_1 = band_range_1
    low_freq_2, high_freq_2 = band_range_2
    
    # Get first channel (can be modified to use different channels)
    ch1 = trial[0, :]
    
    # Filter the same channel for two different frequency bands
    band_1_signal = bandpass_filter(ch1, low_freq_1, high_freq_1, fs=200)
    band_2_signal = bandpass_filter(ch1, low_freq_2, high_freq_2, fs=200)
    
    # Calculate transfer entropy between the filtered band signals
    te_feature = phase_transfer_entropy(band_1_signal, band_2_signal)
    
    return [te_feature]

def process_data_band_to_band_transfer_entropy(data_array, band_range_1, band_range_2):
    """
    Process data array and extract phase transfer entropy features between two frequency bands.
    data_array: shape (trials, channels, samples)
    band_range_1: tuple (low_freq, high_freq) for source band
    band_range_2: tuple (low_freq, high_freq) for target band
    """
    feature_list = []
    for trial_data in data_array:
        trial_data = np.array(trial_data, dtype=np.float64)
        
        # Extract transfer entropy features between the two bands
        trial_features = extract_transfer_entropy_features_between_bands(trial_data, band_range_1, band_range_2)
        feature_list.append(trial_features)
    
    return np.array(feature_list)

def evaluate_band_to_band_transfer_entropy(data_array, labels_array, band_name_1, band_range_1, band_name_2, band_range_2):
    """
    Evaluate classifiers using phase transfer entropy features between two frequency bands.
    """
    print(f"\n=== Evaluating Transfer Entropy: {band_name_1.upper()} → {band_name_2.upper()} ===")

    # Extract transfer entropy features between the two bands
    X = process_data_band_to_band_transfer_entropy(data_array, band_range_1, band_range_2)

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(labels_array)

    # Prepare cross-validation
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    results = {}

    for name, clf in classifiers.items():
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', clf)
        ])

        scores = cross_val_score(pipeline, X, y_encoded, cv=skf, scoring='accuracy')

        results[name] = {
            'Mean Accuracy': np.mean(scores),
            'Std Accuracy': np.std(scores),
            'All Scores': scores
        }

        print(f"{name}: {np.mean(scores):.3f} ± {np.std(scores):.3f}")

    return results

def evaluate_all_band_combinations_transfer_entropy(data_array, labels_array):
    """
    Evaluate phase transfer entropy features for all frequency band combinations.
    """
    print("\n" + "="*80)
    print("EVALUATING Phase TRANSFER ENTROPY BETWEEN ALL FREQUENCY BAND COMBINATIONS")
    print("="*80)
    
    all_results = {}
    
    # Evaluate each band combination (including self-connections and bidirectional)
    for band_name_1, band_range_1 in bands.items():
        for band_name_2, band_range_2 in bands.items():
            if band_name_1 == band_name_2:
                # Skip self-connections (e.g., alpha to alpha)
                continue
            combination_key = f"{band_name_1}_to_{band_name_2}"
            band_results = evaluate_band_to_band_transfer_entropy(
                data_array, labels_array, band_name_1, band_range_1, band_name_2, band_range_2
            )
            all_results[combination_key] = band_results
    
    # Create comprehensive visualization
    create_comprehensive_transfer_entropy_plots(all_results)
    
    # Print summary table
    create_comprehensive_summary_table(all_results)
    
    return all_results

def create_comprehensive_transfer_entropy_plots(all_results):
    """
    Create comprehensive plots comparing phase transfer entropy performance across band combinations and classifiers.
    """
    # Prepare data for plotting
    plot_data = []
    for combination_name, combination_results in all_results.items():
        source_band, target_band = combination_name.split('_to_')
        for classifier_name, metrics in combination_results.items():
            plot_data.append({
                'Source Band': source_band.title(),
                'Target Band': target_band.title(),
                'Combination': f"{source_band.title()} → {target_band.title()}",
                'Classifier': classifier_name,
                'Mean Accuracy': metrics['Mean Accuracy'],
                'Std Accuracy': metrics['Std Accuracy']
            })
    
    df = pd.DataFrame(plot_data)
    
    # Create comprehensive figure with multiple subplots
    fig = plt.figure(figsize=(24, 16))
    
    # Main title
    fig.suptitle('Phase Transfer Entropy Classification Performance: Band-to-Band Analysis', 
                 fontsize=20, fontweight='bold', y=0.98)
    
    # Create grid layout
    gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
    
    # Plot 1: Overall accuracy heatmap by combination
    ax1 = fig.add_subplot(gs[0, :2])
    pivot_accuracy = df.groupby(['Source Band', 'Target Band'])['Mean Accuracy'].mean().unstack()
    sns.heatmap(pivot_accuracy, annot=True, fmt='.3f', cmap='viridis', ax=ax1, cbar_kws={'label': 'Mean Accuracy'})
    ax1.set_title('Mean Accuracy Heatmap: Source → Target Band Combinations', fontweight='bold')
    ax1.set_xlabel('Target Band')
    ax1.set_ylabel('Source Band')
    
    # Plot 2: Best classifier per combination
    ax2 = fig.add_subplot(gs[0, 2:])
    best_per_combination = df.loc[df.groupby('Combination')['Mean Accuracy'].idxmax()]
    best_per_combination_sorted = best_per_combination.nlargest(15, 'Mean Accuracy')
    
    sns.barplot(data=best_per_combination_sorted, y='Combination', x='Mean Accuracy', 
                hue='Classifier', ax=ax2, palette='Set2')
    ax2.set_title('Top 15 Band Combinations (Best Classifier Each)', fontweight='bold')
    ax2.set_xlabel('Mean Accuracy')
    ax2.set_ylabel('Band Combination')
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Plot 3: Classifier performance comparison
    ax3 = fig.add_subplot(gs[1, :2])
    classifier_performance = df.groupby('Classifier')['Mean Accuracy'].agg(['mean', 'std']).reset_index()
    classifier_performance = classifier_performance.sort_values('mean', ascending=True)
    
    bars = ax3.barh(classifier_performance['Classifier'], classifier_performance['mean'], 
                    xerr=classifier_performance['std'], capsize=5, color='skyblue', alpha=0.7)
    ax3.set_title('Average Classifier Performance Across All Band Combinations', fontweight='bold')
    ax3.set_xlabel('Mean Accuracy')
    ax3.set_ylabel('Classifier')
    ax3.grid(axis='x', alpha=0.3)
    
    # Add value labels on bars
    for i, (bar, mean_val) in enumerate(zip(bars, classifier_performance['mean'])):
        ax3.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, 
                f'{mean_val:.3f}', ha='left', va='center', fontweight='bold')
    
    # Plot 4: Source band analysis
    ax4 = fig.add_subplot(gs[1, 2:])
    source_performance = df.groupby('Source Band')['Mean Accuracy'].agg(['mean', 'std']).reset_index()
    source_performance = source_performance.sort_values('mean', ascending=False)
    
    sns.barplot(data=source_performance, x='Source Band', y='mean', ax=ax4, palette='viridis')
    ax4.errorbar(range(len(source_performance)), source_performance['mean'], 
                yerr=source_performance['std'], fmt='none', color='black', capsize=5)
    ax4.set_title('Average Performance by Source Band', fontweight='bold')
    ax4.set_xlabel('Source Band')
    ax4.set_ylabel('Mean Accuracy')
    ax4.grid(axis='y', alpha=0.3)
    
    # Plot 5: Distribution of accuracies
    ax5 = fig.add_subplot(gs[2, :2])
    sns.violinplot(data=df, x='Classifier', y='Mean Accuracy', ax=ax5, palette='Set3')
    ax5.set_title('Distribution of Accuracies by Classifier', fontweight='bold')
    ax5.set_xlabel('Classifier')
    ax5.set_ylabel('Mean Accuracy')
    ax5.tick_params(axis='x', rotation=45)
    
    # Plot 6: Top performing combinations detailed view
    ax6 = fig.add_subplot(gs[2, 2:])
    top_combinations = df.nlargest(20, 'Mean Accuracy')
    
    # Create a scatter plot with different colors for each classifier
    for classifier in top_combinations['Classifier'].unique():
        classifier_data = top_combinations[top_combinations['Classifier'] == classifier]
        ax6.scatter(classifier_data['Mean Accuracy'], classifier_data['Combination'], 
                   label=classifier, alpha=0.7, s=60)
    
    ax6.set_title('Top 20 Performances: Combination vs Accuracy', fontweight='bold')
    ax6.set_xlabel('Mean Accuracy')
    ax6.set_ylabel('Band Combination')
    ax6.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax6.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def create_comprehensive_summary_table(all_results):
    """
    Create and print a comprehensive summary table for phase transfer entropy results.
    """
    print("\n" + "="*120)
    print("PHASE TRANSFER ENTROPY BETWEEN FREQUENCY BANDS - COMPREHENSIVE SUMMARY")
    print("="*120)
    
    # Create summary dataframe
    summary_data = []
    for combination_name, combination_results in all_results.items():
        source_band, target_band = combination_name.split('_to_')
        for classifier_name, metrics in combination_results.items():
            summary_data.append({
                'Source Band': source_band.title(),
                'Target Band': target_band.title(),
                'Combination': f"{source_band.title()} → {target_band.title()}",
                'Classifier': classifier_name,
                'Mean Accuracy': metrics['Mean Accuracy'],
                'Std Accuracy': metrics['Std Accuracy'],
                'Performance': f"{metrics['Mean Accuracy']:.3f} ± {metrics['Std Accuracy']:.3f}"
            })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Print top 20 performances
    print("TOP 20 PERFORMANCES:")
    print("-" * 120)
    top_20 = summary_df.nlargest(20, 'Mean Accuracy')[['Combination', 'Classifier', 'Performance']]
    print(top_20.to_string(index=False))
    
    # Print best performance per band combination
    print("\n" + "="*80)
    print("BEST CLASSIFIER PER BAND COMBINATION:")
    print("="*80)
    
    best_per_combination = summary_df.loc[summary_df.groupby('Combination')['Mean Accuracy'].idxmax()]
    best_per_combination_sorted = best_per_combination.sort_values('Mean Accuracy', ascending=False)
    
    for _, row in best_per_combination_sorted.head(15).iterrows():
        print(f"{row['Combination']:20s}: {row['Classifier']:18s} - {row['Performance']}")
    
    # Print best performance per classifier
    print("\n" + "="*80)
    print("BEST BAND COMBINATION PER CLASSIFIER:")
    print("="*80)
    
    best_per_classifier = summary_df.loc[summary_df.groupby('Classifier')['Mean Accuracy'].idxmax()]
    best_per_classifier_sorted = best_per_classifier.sort_values('Mean Accuracy', ascending=False)
    
    for _, row in best_per_classifier_sorted.iterrows():
        print(f"{row['Classifier']:18s}: {row['Combination']:20s} - {row['Performance']}")
    
    # Print overall statistics
    print("\n" + "="*80)
    print("OVERALL STATISTICS:")
    print("="*80)
    
    overall_best = summary_df.loc[summary_df['Mean Accuracy'].idxmax()]
    print(f"Best Overall Performance: {overall_best['Combination']} with {overall_best['Classifier']} - {overall_best['Performance']}")
    print(f"Average Accuracy Across All Combinations: {summary_df['Mean Accuracy'].mean():.3f} ± {summary_df['Mean Accuracy'].std():.3f}")
    print(f"Median Accuracy: {summary_df['Mean Accuracy'].median():.3f}")
    print(f"Range: {summary_df['Mean Accuracy'].min():.3f} - {summary_df['Mean Accuracy'].max():.3f}")

def process_data(data_array, feature_extractor):
    """
    Legacy function - process data using full bandwidth.
    """
    features = []
    for trial in data_array:
        trial = np.array(trial, dtype=np.float64)
        # Apply bandpass filter to entire trial
        if trial.ndim == 2:
            filtered_signal = np.array([bandpass_filter(trial[i], fs=200) for i in range(trial.shape[0])])
        else:
            filtered_signal = bandpass_filter(trial, fs=200)
        feat = feature_extractor(filtered_signal)
        features.append(feat)
    return np.array(features)

def create_legacy_performance_plot(results):
    """
    Create an improved plot for legacy full-bandwidth results.
    """
    accuracy_df = pd.DataFrame({
        'Classifier': list(results.keys()),
        'Mean Accuracy': [results[m]['Mean Accuracy'] for m in results],
        'Std Accuracy': [results[m]['Std Accuracy'] for m in results]
    }).sort_values(by='Mean Accuracy', ascending=False)

    plt.figure(figsize=(12, 8))
    
    # Create bar plot with error bars
    bars = plt.bar(accuracy_df['Classifier'], accuracy_df['Mean Accuracy'], 
                   yerr=accuracy_df['Std Accuracy'], capsize=5, 
                   color='steelblue', alpha=0.7, edgecolor='navy')

    # Add value labels on bars
    for bar, (_, row) in zip(bars, accuracy_df.iterrows()):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + row['Std Accuracy'] + 0.01,
                f'{row["Mean Accuracy"]:.3f}±{row["Std Accuracy"]:.3f}', 
                ha='center', va='bottom', fontweight='bold')

    plt.title("10-Fold Cross-Validated Accuracy using Phase Transfer Entropy Features\n(Full Bandwidth Analysis)", 
              fontsize=14, fontweight='bold')
    plt.ylabel("Accuracy", fontsize=12)
    plt.xlabel("Classifier", fontsize=12)
    plt.ylim(0, min(1.0, accuracy_df['Mean Accuracy'].max() + accuracy_df['Std Accuracy'].max() + 0.1))
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

# ============================================================================
# USAGE EXAMPLES:
# ============================================================================

print("Transfer Entropy Band Analysis Functions Ready!")
print("\nAvailable Functions:")
print("="*50)
print("1. evaluate_all_band_combinations_transfer_entropy(data_array, labels_array)")
print("   - Analyzes transfer entropy between ALL frequency band combinations")
print("   - Includes comprehensive visualizations and summary tables")
print("")
print("2. evaluate_features(data_array, labels_array)")
print("   - Legacy full bandwidth analysis")
print("   - Uses transfer entropy between channels or filtered versions")
print("")
print("3. evaluate_band_to_band_transfer_entropy(data_array, labels_array, 'alpha', (8, 14), 'beta', (14, 30))")
print("   - Analyzes specific band-to-band transfer entropy")
print("")
print("Required data format:")
print("- data_array: shape (trials, channels, samples)")
print("- labels_array: shape (trials,)")
print("")
print("Note: The transfer_entropy(x, y) function is included as a placeholder.")
print("      Replace it with your actual transfer entropy implementation!")
print("")
print("Example usage:")
print("results = evaluate_all_band_combinations_transfer_entropy(data_array, labels_array)")

Transfer Entropy Band Analysis Functions Ready!

Available Functions:
1. evaluate_all_band_combinations_transfer_entropy(data_array, labels_array)
   - Analyzes transfer entropy between ALL frequency band combinations
   - Includes comprehensive visualizations and summary tables

2. evaluate_features(data_array, labels_array)
   - Legacy full bandwidth analysis
   - Uses transfer entropy between channels or filtered versions

3. evaluate_band_to_band_transfer_entropy(data_array, labels_array, 'alpha', (8, 14), 'beta', (14, 30))
   - Analyzes specific band-to-band transfer entropy

Required data format:
- data_array: shape (trials, channels, samples)
- labels_array: shape (trials,)

Note: The transfer_entropy(x, y) function is included as a placeholder.
      Replace it with your actual transfer entropy implementation!

Example usage:
results = evaluate_all_band_combinations_transfer_entropy(data_array, labels_array)


In [8]:
evaluate_all_band_combinations_transfer_entropy(data_array, labels_array)


EVALUATING Phase TRANSFER ENTROPY BETWEEN ALL FREQUENCY BAND COMBINATIONS

=== Evaluating Transfer Entropy: DELTA → THETA ===


KeyboardInterrupt: 